# An Introduction to LlamaIndex Query Pipelines

## Overview
LlamaIndex provides a declarative query API that allows you to chain together different modules in order to orchestrate simple-to-advanced workflows over your data.

This is centered around our `QueryPipeline` abstraction. Load in a variety of modules (from LLMs to prompts to retrievers to other pipelines), connect them all together into a sequential chain or DAG, and run it end2end.

**NOTE**: You can orchestrate all these workflows without the declarative pipeline abstraction (by using the modules imperatively and writing your own functions). So what are the advantages of `QueryPipeline`? 

- Express common workflows with fewer lines of code/boilerplate
- Greater readability
- Greater parity / better integration points with common low-code / no-code solutions (e.g. LangFlow)
- [In the future] A declarative interface allows easy serializability of pipeline components, providing portability of pipelines/easier deployment to different systems.

## Cookbook

In this cookbook we give you an introduction to our `QueryPipeline` interface and show you some basic workflows you can tackle.

- Chain together prompt and LLM
- Chain together query rewriting (prompt + LLM) with retrieval
- Chain together a full RAG query pipeline (query rewriting, retrieval, reranking, response synthesis)
- Setting up a custom query component

## Setup

Here we setup some data + indexes (from PG's essay) that we'll be using in the rest of the cookbook.

In [ ]:
# setup Arize Phoenix for logging/observability
import phoenix as px

px.launch_app()
import llama_index

llama_index.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from llama_index.query_pipeline.query import QueryPipeline
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    SimpleDirectoryReader,
    load_index_from_storage,
)

In [ ]:
reader = SimpleDirectoryReader("../data/paul_graham")

In [ ]:
docs = reader.load_data()

In [ ]:
import os
from llama_index.storage import StorageContext

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

## 1. Chain Together Prompt and LLM 

In this section we show a super simple workflow of chaining together a prompt with LLM.

We simply define `chain` on initialization. This is a special case of a query pipeline where the components are purely sequential, and we automatically convert outputs into the right format for the next inputs.

In [ ]:
# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

In [ ]:
output = p.run(movie_name="The Departed")

> Running module 6d362a50-c478-47af-9855-29e8d108784b with input: 
movie_name: The Departed

> Running module 45856814-f25b-437a-a1c2-faf0b6ad0f59 with input: 
messages: Please generate related movies to The Departed



In [ ]:
print(str(output))

assistant: 1. Infernal Affairs (2002) - This is the original Hong Kong film that inspired The Departed. It follows a similar storyline of undercover cops infiltrating the criminal underworld.

2. Internal Affairs (1990) - This American crime thriller, starring Richard Gere and Andy Garcia, revolves around a corrupt cop and an internal affairs officer determined to expose him.

3. The Town (2010) - Directed by and starring Ben Affleck, this crime drama focuses on a group of bank robbers in Boston and the FBI agent determined to bring them down.

4. Heat (1995) - Directed by Michael Mann, this classic crime film features Al Pacino and Robert De Niro as a detective and a professional thief, respectively, engaged in a cat-and-mouse game.

5. Training Day (2001) - Denzel Washington won an Academy Award for his role as a corrupt narcotics detective who takes a rookie cop, played by Ethan Hawke, on a dangerous ride-along through the streets of Los Angeles.

6. The Departed (2006) - Although a

### Try Output Parsing

Let's parse the outputs into a structured Pydantic object.

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from llama_index.output_parsers import PydanticOutputParser


class Movie(BaseModel):
    """Object representing a single movie."""

    name: str = Field(..., description="Name of the movie.")
    year: int = Field(..., description="Year of the movie.")


class Movies(BaseModel):
    """Object representing a list of movies."""

    movies: List[Movie] = Field(..., description="List of movies.")


llm = OpenAI(model="gpt-3.5-turbo")
output_parser = PydanticOutputParser(Movies)
prompt_str = """\
Please generate related movies to {movie_name}. Output with the following JSON format: 
"""
prompt_str = output_parser.format(prompt_str)
# prompt_str = prompt_str + output_parser.format_string

In [ ]:
prompt_str.format(movie_name="test")

'Please generate related movies to test. Output with the following JSON format: \n\n\n\nHere\'s a JSON schema to follow:\n{"title": "Movies", "description": "Object representing a list of movies.", "type": "object", "properties": {"movies": {"title": "Movies", "description": "List of movies.", "type": "array", "items": {"$ref": "#/definitions/Movie"}}}, "required": ["movies"], "definitions": {"Movie": {"title": "Movie", "description": "Object representing a single movie.", "type": "object", "properties": {"name": {"title": "Name", "description": "Name of the movie.", "type": "string"}, "year": {"title": "Year", "description": "Year of the movie.", "type": "integer"}}, "required": ["name", "year"]}}}\n\nOutput a valid JSON object but do not repeat the schema.\n'

In [ ]:
# add JSON spec to prompt template
prompt_tmpl = PromptTemplate(prompt_str)

p = QueryPipeline(chain=[prompt_tmpl, llm, output_parser], verbose=True)
output = p.run(movie_name="Toy Story")

> Running module f95f99e3-41c1-470e-955a-f1ebbf1abc59 with input: 
movie_name: Toy Story

> Running module b9f193fe-4798-4d86-a335-dd3ed0dc63b2 with input: 
messages: Please generate related movies to Toy Story. Output with the following JSON format: 



Here's a JSON schema to follow:
{"title": "Movies", "description": "Object representing a list of movies.", "typ...

> Running module 5342cd41-055c-4554-aed5-f23ced25195d with input: 
input: assistant: {
  "movies": [
    {
      "name": "Finding Nemo",
      "year": 2003
    },
    {
      "name": "Monsters, Inc.",
      "year": 2001
    },
    {
      "name": "Cars",
      "year": 2006
...



In [ ]:
output

Movies(movies=[Movie(name='Finding Nemo', year=2003), Movie(name='Monsters, Inc.', year=2001), Movie(name='Cars', year=2006), Movie(name='The Incredibles', year=2004), Movie(name='Ratatouille', year=2007)])

## Chain Together Query Rewriting Workflow (prompts + LLM) with Retrieval

Here we try a slightly more complex workflow where we send the input through two prompts before initiating retrieval.

1. Generate question about given topic.
2. Hallucinate answer given question, for better retrieval.

Since each prompt only takes in one input, note that the `QueryPipeline` will automatically chain LLM outputs into the prompt and then into the LLM. 

You'll see how to define links more explicitly in the next section.

In [ ]:
from llama_index.postprocessor import CohereRerank

# generate question regarding topic
prompt_str1 = "Please generate a concise question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)
# use HyDE to hallucinate answer.
prompt_str2 = (
    "Please write a passage to answer the question\n"
    "Try to include as many key details as possible.\n"
    "\n"
    "\n"
    "{query_str}\n"
    "\n"
    "\n"
    'Passage:"""\n'
)
prompt_tmpl2 = PromptTemplate(prompt_str2)

llm = OpenAI(model="gpt-3.5-turbo")
retriever = index.as_retriever(similarity_top_k=5)
p = QueryPipeline(
    chain=[prompt_tmpl1, llm, prompt_tmpl2, llm, retriever], verbose=True
)

In [ ]:
nodes = p.run(topic="college")
len(nodes)

> Running module 1f43ab6e-be00-4663-bee3-030372f7e4ac with input: 
topic: college

> Running module e9f2840c-4a06-4cc7-9c1b-956ae33e3db9 with input: 
messages: Please generate a concise question about Paul Graham's life regarding the following topic college

> Running module 4b1d1ebd-d54b-4d7b-bfbd-9826b1017475 with input: 
query_str: assistant: How did Paul Graham's college experience shape his career and entrepreneurial mindset?

> Running module 296fccf9-44df-41df-a233-d8a69645c391 with input: 
messages: Please write a passage to answer the question
Try to include as many key details as possible.


assistant: How did Paul Graham's college experience shape his career and entrepreneurial mindset?


Pass...

> Running module eefd3d5a-625d-4780-90f3-057c20d4b2dd with input: 
input: assistant: Paul Graham's college experience played a pivotal role in shaping his career and entrepreneurial mindset. As a student at Cornell University, Graham immersed himself in the world of compute...



5

## Create a Full RAG Pipeline as a DAG

Here we chain together a full RAG pipeline consisting of query rewriting, retrieval, reranking, and response synthesis.

Here we can't use `chain` syntax because certain modules depend on multiple inputs (for instance, response synthesis expects both the retrieved nodes and the original question). Instead we'll construct a DAG explicitly, through `add_modules` and then `add_link`.

In [ ]:
from llama_index.postprocessor import CohereRerank
from llama_index.response_synthesizers import TreeSummarize
from llama_index import ServiceContext

# define modules
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")
retriever = index.as_retriever(similarity_top_k=3)
reranker = CohereRerank()
summarizer = TreeSummarize(
    service_context=ServiceContext.from_defaults(llm=llm)
)

In [ ]:
# define query pipeline
p = QueryPipeline(verbose=True)
p.add_modules(
    {
        "llm": llm,
        "prompt_tmpl": prompt_tmpl,
        "retriever": retriever,
        "summarizer": summarizer,
        "reranker": reranker,
    }
)

Next we draw links between modules with `add_link`. `add_link` takes in the source/destination module ids, and optionally the `source_key` and `dest_key`. Specify the `source_key` or `dest_key` if there are multiple outputs/inputs respectively.

You can view the set of input/output keys for each module through `module.as_query_component().input_keys` and `module.as_query_component().output_keys`. 

Here we explicitly specify `dest_key` for the `reranker` and `summarizer` modules because they take in two inputs (query_str and nodes). 

In [ ]:
p.add_link("prompt_tmpl", "llm")
p.add_link("llm", "retriever")
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
p.add_link("llm", "summarizer", dest_key="query_str")

# look at summarizer input keys
print(summarizer.as_query_component().input_keys)

required_keys={'nodes', 'query_str'} optional_keys=set()


In [ ]:
response = p.run(topic="YC")

> Running module prompt_tmpl with input: 
topic: YC

> Running module llm with input: 
messages: Please generate a question about Paul Graham's life regarding the following topic YC

> Running module retriever with input: 
input: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?

> Running module reranker with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='bf639f66-34a2-46d2-a30d-9aff96d130b9', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file...

> Running module summarizer with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='7768a74a-5a54-414f-9217-47f976da7891', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'fi

In [ ]:
print(str(response))

Paul Graham played a significant role in the founding and development of Y Combinator (YC). He was one of the co-founders of YC and was actively involved in its early stages. He helped establish the Summer Founders Program (SFP) and was responsible for selecting and funding the initial batch of startups. Graham also played a key role in shaping the funding model for YC, based on previous deals and experiences. As YC grew, Graham's involvement shifted, and he focused more on writing essays and working on YC, while also being involved in other projects such as Hacker News. However, he eventually decided to hand over the reins of YC to Sam Altman and retire from his role in the organization.


In [ ]:
# you can do async too
response = await p.arun(topic="YC")
print(str(response))

> Running module prompt_tmpl with input: 
topic: YC

> Running module llm with input: 
messages: Please generate a question about Paul Graham's life regarding the following topic YC

> Running module retriever with input: 
input: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?

> Running module reranker with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='bf639f66-34a2-46d2-a30d-9aff96d130b9', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file...

> Running module summarizer with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='7768a74a-5a54-414f-9217-47f976da7891', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'fi

## Defining a Custom Component in a Query Pipeline

You can easily define a custom component. Simply subclass a `QueryComponent`, implement validation/run functions + some helpers, and plug it in.

Let's wrap the related movie generation prompt+LLM chain from the first example into a custom component.

In [ ]:
from llama_index.query_pipeline import (
    CustomQueryComponent,
    InputKeys,
    OutputKeys,
)
from typing import Dict, Any
from llama_index.llms.llm import BaseLLM
from pydantic import Field


class RelatedMovieComponent(CustomQueryComponent):
    """Related movie component."""

    llm: BaseLLM = Field(..., description="OpenAI LLM")

    def _validate_component_inputs(
        self, input: Dict[str, Any]
    ) -> Dict[str, Any]:
        """Validate component inputs during run_component."""
        # NOTE: this is OPTIONAL but we show you here how to do validation as an example
        return input

    @property
    def _input_keys(self) -> set:
        """Input keys dict."""
        # NOTE: These are required inputs. If you have optional inputs please override
        # `optional_input_keys_dict`
        return {"movie"}

    @property
    def _output_keys(self) -> set:
        return {"output"}

    def _run_component(self, **kwargs) -> Dict[str, Any]:
        """Run the component."""
        # use QueryPipeline itself here for convenience
        prompt_str = "Please generate related movies to {movie_name}"
        prompt_tmpl = PromptTemplate(prompt_str)
        p = QueryPipeline(chain=[prompt_tmpl, llm])
        return {"output": p.run(movie_name=kwargs["movie"])}

Let's try the custom component out! We'll also add a step to convert the output to Shakespeare.

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")
component = RelatedMovieComponent(llm=llm)

# let's add some subsequent prompts for fun
prompt_str = """\
Here's some text:

{text}

Can you rewrite this in the voice of Shakespeare?
"""
prompt_tmpl = PromptTemplate(prompt_str)

p = QueryPipeline(chain=[component, prompt_tmpl, llm], verbose=True)

In [ ]:
output = p.run(movie="Love Actually")

> Running module 2f97b71b-4b62-40c3-8848-dc175d8c6393 with input: 
movie: Love Actually

> Running module bfc5d991-ac65-456b-aece-105d7bdb2087 with input: 
text: assistant: 1. "Valentine's Day" (2010) - A star-studded ensemble cast explores interconnected love stories on Valentine's Day in Los Angeles.

2. "New Year's Eve" (2011) - Similar to "Love Actually," ...

> Running module 0eb499e2-7228-48cf-8ec1-d63ced7f29c9 with input: 
messages: Here's some text:

assistant: 1. "Valentine's Day" (2010) - A star-studded ensemble cast explores interconnected love stories on Valentine's Day in Los Angeles.

2. "New Year's Eve" (2011) - Similar t...



In [ ]:
print(str(output))

assistant: 1. "Valentine's Daye" (2010) - A troupe of stars doth explore interconnected tales of love on Valentine's Daye in fair Los Angeles.

2. "New Year's Eve" (2011) - Much like "Love Actually," this play doth followeth diverse characters as they navigate love and relationships on New Year's Eve in the grand city of New York.

3. "He's Just Not That Into Thee" (2009) - This romantic comedy doth intertwine the lives of several souls as they endeavor to comprehend the complexities of modern relationships.

4. "Crazy, Stupid, Love" (2011) - A man of middle age doth witness his life unravel when his wife doth request a divorce, leading him to seeketh guidance from a young bachelor who doth aid him in rediscovering love.

5. "The Holiday" (2006) - Two women, one from fair Los Angeles and the other from England, doth exchange homes during the Christmas season and unexpectedly findeth love in their new surroundings.

6. "Four Weddings and a Funeral" (1994) - This British romantic comedy 